[참고- [파이썬]KoBERT로 다중 분류 모델 만들기 - 코드](https://velog.io/@seolini43/KOBERT로-다중-분류-모델-만들기-파이썬Colab)

In [1]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers
# !pip install torch
# !pip install datasets
# !pip install kobert
# !pip install -U huggingface_hub

## 모듈 불러오기

In [2]:
import torch
import numpy as np
import random
from emotionModule import load, modelEngine

In [3]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## 모델 불러오기

In [4]:
loader = load.loader()

In [5]:
# MODEL_NAME = "klue/bert-base"
MODEL_NAME = 'kobert'

bertmodel = loader.BertModel(MODEL_NAME)
tokenizer, vocab = loader.tokenizer(MODEL_NAME)

using cached model. /home/work/data/final/.cache/kobert_v1.zip
using cached model. /home/work/data/final/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/work/data/final/.cache/kobert_v1.zip
using cached model. /home/work/data/final/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/work/data/final/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 데이터셋 불러오기

In [6]:
DATA_PATH = "KOTE_relabel/"
ds = loader.dataset(DATA_PATH)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'class_idx', 'class_label'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['text', 'class_idx', 'class_label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'class_idx', 'class_label'],
        num_rows: 5000
    })
})

### 모델 configuration

In [7]:
# set config
label_dict = dict(zip(range(5), ["행복한", "화나는", "평온한", "슬픈", "없음"]))

configs = {"max_len": 256,
"batch_size" : 16,
"warmup_ratio": 0.1,
"num_epochs": 25,
"max_grad_norm": 1,
"log_interval": 200,
"learning_rate":  5e-5,
"num_classes" :len(label_dict),
"train_len" : len(ds["train"])
}

In [8]:
preprocessor = load.preprocess(tokenizer = tokenizer, vocab = vocab, configs = configs)

In [9]:
tok_train = preprocessor.tokenize(ds["train"])
tok_test = preprocessor.tokenize(ds["test"])
tok_valid = preprocessor.tokenize(ds["validation"])

In [10]:
train = preprocessor.cvt2dataloader(tok_train)
test = preprocessor.cvt2dataloader(tok_test)
valid = preprocessor.cvt2dataloader(tok_valid)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [12]:
engine = modelEngine.engine(bertmodel, device, configs)

In [13]:
# compiling as default setting
engine.set_default(configs)

/home/work/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# # training
# engine.fit(train, valid, configs)

## 모델 저장

In [14]:
# MODEL_PATH = "model/KoBert_full"

In [577]:
# torch.save(engine.model.state_dict(), MODEL_PATH)

## 학습한 모델 불러오기

In [15]:
MODEL_PATH = "model/KoBert_full"

In [16]:
new_engine = modelEngine.engine(bertmodel, device, configs)

In [17]:
new_engine.model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

### 가사 validation set 불러오기

In [18]:
VALID_PATH = "lyrics_emotion"
valid_ds = loader.dataset(VALID_PATH)
valid_ds

Dataset({
    features: ['genre', 'lyrics', 'emotion', 'idx'],
    num_rows: 113
})

### 감정 분류

In [19]:
predictions, probs = new_engine.predict(new_text = valid_ds["lyrics"], preprocessor = preprocessor, return_max_only=True)

In [20]:
results = valid_ds.to_pandas()
results["prediction"] = [label_dict[idx] for idx in predictions]

In [21]:
results

,genre,lyrics,emotion,idx,prediction
0,댄스,20 22 PSY Coming back (이리 오너라) Long time no se...,행복한,0,행복한
1,록/메탈,Look at you 넌 못 감당해 날 Ya took off hook 기분은 Cok...,화나는,1,행복한
2,발라드,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,행복한,0,슬픈
3,발라드,너를 위해 해 줄 것이 하나 없어서 보낼 수밖에 없었고 네가 없이 사는 법을 알지 ...,슬픈,3,슬픈
4,발라드,하얀 머리 뽑아 달라며 한 개 백 원이라던 그 시절 다 지나가고 이젠 흰 눈만 남았...,슬픈,3,슬픈
...,...,...,...,...,...
108,발라드,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,행복한,0,슬픈
109,발라드,둥근 해가 뜨면 제일 먼저 기분 좋은 상상을 하지 하나 둘 셋 자리에 일어나 하마처...,행복한,0,행복한
110,인디음악,사랑노래가 너무 많아서 어떤 노래로 고백을 할까 처음본 순간 falling love...,행복한,0,행복한
111,국내뮤지컬,잔인한 곳 무자비한 곳 목구멍 풀칠해 버텨내 살아내는 것도 벅차 세상은 잔인한 곳 ...,화나는,1,화나는


### 정확도

In [22]:
(results["emotion"] == results["prediction"]).mean()

0.5398230088495575